In [1]:
# imports

import pandas as pd
import numpy as np
import re

from helper.data_cleaning import find_highest_value, extract_cpv, convert_to_euro

# Reading and cleaning parsed data. 

This notebook reads parsed tender data and cleans the dataset.
It creates the "parsed_data_new.csv" file. 
Saved dataset is further processed in the 02-descriptives notebook.

## Reading data

In [2]:
columns = ["id", "title", "cpv", "contract_type", "description", "country", "contracting_authority_type", "contracting_authority_activity", "value", "type_of_procedure", 
           "government_procurement_agreement", "framework_agreement", "notice_dispatch_date", "application_date", "tender_opening_date"]

# read data
df = pd.read_csv('../data/parsed_data_new.csv', encoding='utf-8', names=columns, low_memory=False)

print(f"There are a total of {len(df)} tenders in the dataset.")

df.head()

There are a total of 349590 tenders in the dataset.


,id,title,cpv,contract_type,description,country,contracting_authority_type,contracting_authority_activity,value,type_of_procedure,government_procurement_agreement,framework_agreement,notice_dispatch_date,application_date,tender_opening_date
0,1-2021,Cyber Threat Intelligence Infrastructure – Des...,"72000000 IT services: consulting, software dev...",Services,ENISA seeks to contract service providers for ...,"<div class=""txtmark"" style=""color:black"">Offic...",European institution/agency or international o...,Other activity: Cybersecurity,Value excluding VAT: 3 150 000.00 EUR,Open procedure,The procurement is covered by the Government P...,The procurement involves the establishment of ...,23/12/2020,NaN,NaN
1,1000-2015,Ds-rempart.,72910000 Computer back-up services,ServicesService category No 7: Computer and re...,Maîtrise d'oeuvre du secours informatique de l...,"<div class=""txtmark"" style=""color:black""><p cl...",Ministry or any other national or federal auth...,General public services,Lowest offer 1 095 202 and highest offer 2 205...,Open,The contract is covered by the Government Proc...,No,30.12.2014,NaN,NaN
2,100000-2015,"Service de téléphonie filaire, de télécommunic...",64210000 Telephone and data transmission servi...,ServicesService category No 5: Telecommunicati...,"Services de téléphonie filaire, de télécommuni...","<div class=""txtmark"" style=""color:black""><p cl...",Regional or local authority,Environment,NaN,Open,The contract is covered by the Government Proc...,The notice involves the establishment of a fra...,16.3.2015,27.4.2015 - 12:00,Persons authorised to be present at the openin...
3,100001-2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100004-2015,NaN,"48810000 Information systems, 48000000 Softwar...",ServicesService category No 7: Computer and re...,Les prestations objet de la consultation conce...,"<div class=""txtmark"" style=""color:black""><p cl...",Body governed by public law,Environment,Ce marché sera conclu sous la forme d'un march...,Open,The contract is covered by the Government Proc...,The notice involves a public contract,17.3.2015,27.4.2015 - 16:00,NaN


In [3]:
# cleaning df

# Value. Since the value column contains a lot of information, we need to extract the currency and the highest value from the column.

european_currencies = "EUR|GBP|CZK|HRK|DKK|PLN|RON|SEK|CHF|ISK|NOK|HUF|BAM|MKD|ALL|UAH|MDL|RSD|BGN"
df['currency'] = df['value'].replace(np.nan, "").apply(lambda x: re.findall(european_currencies, x)[0] if re.findall(european_currencies, x) else None)
df['value_hi'] = df['value'].apply(find_highest_value)

# CPV. Extract all numbers from column cpv and save them in a list.
df["cpv"] = df["cpv"].apply(extract_cpv)

# Year. Extract year from notice_dispatch_date, application_date, tender_opening_date and id.

df["notice_dispatch_date"] = df["notice_dispatch_date"].str.extract('(\d{4})')
df["application_date"] = df["application_date"].str.extract('(\d{4})')
df["tender_opening_date"] = df["tender_opening_date"].str.extract('(\d{4})')
df["id-year"] = df["id"].str.split("-").str[1]
df["year"] = df["notice_dispatch_date"].fillna(df["application_date"]).fillna(df["tender_opening_date"]).fillna(df["id-year"])

# Contracting authority type.

df["contracting_authority_type"] = df["contracting_authority_type"].str.replace('[^\w\s]','').str.replace('\n', '')
df["contracting_authority_type"] = df["contracting_authority_type"].str.replace('Other.*', 'Other', regex=True)
df["contracting_authority_type"] = df["contracting_authority_type"].str.replace('Ministry or any other national or federal authority.*', 
                                                                                'Ministry or any other national or federal authority', regex=True)
df["contracting_authority_type"] = df["contracting_authority_type"].str.replace('European institution.*|European Institution.*', 
                                                                                'European institution/agency or international organisation', regex=True)
df["contracting_authority_type"] = df["contracting_authority_type"].str.replace('agencyoffice', 'agency/office', regex=True)


# Contacting authority activity.

df["contracting_authority_activity"] = df["contracting_authority_activity"].str.replace('Other.*', 'Other', regex=True)
df["contracting_authority_activity"] = df["contracting_authority_activity"].str.replace('[^\w\s]','').str.replace('\n', '')
#split string in contracting_authority_activity column by capital letter and save the first word in a new column
df["contracting_authority_activity"] = df["contracting_authority_activity"].str.split('(?=[A-Z])').str[1]


# Government procurement agreement.

df["government_procurement_agreement"] = df["government_procurement_agreement"].str.replace('.*yes?.|.*Yes?.', 'Yes', regex=True)
df["government_procurement_agreement"] = df["government_procurement_agreement"].str.replace('.*no?.|.*No?.', 'No', regex=True)


# Recoding different types of proceduresin dummy variables - and keeping NaNs.

df["is_framework_agreement"] = np.where(df['framework_agreement'].isna(), None, 
                                        np.where(df['framework_agreement'].str.contains("framework agreement"), "Yes", "No"))


df["is_dynamic"] = np.where(df['framework_agreement'].isna(), None, 
                                        np.where(df['framework_agreement'].str.contains("dynamic purchasing system"), "Yes", "No"))

df["is_open"] = np.where(df['type_of_procedure'].isna(), None, 
                                        np.where(df['type_of_procedure'].str.contains("Open"), "Yes", "No"))


# recode variable "contact_type". Where str.contains "Servic" recode to "Services", where str.contains "Suppli" recode to "Supplies", where str.contains 
# "Works" recode to "Works". If none of the above, recode to np.nan. Save as column contract_type.

df["contract_type"] = np.where(df['contract_type'].str.contains("Servic"), "Services", 
                                 np.where(df['contract_type'].str.contains("Suppli"), "Supplies",
                                            np.where(df['contract_type'].str.contains("Works"), "Works", None)))


# Getting information on buyer.

df['buyer_name'] = df['country'].str.extract(r'Official name: (.*?)<br/>', expand=False)
df['buyer_address'] = df['country'].str.extract(r'Postal address: (.*?)<br/>', expand=False)
df['buyer_town'] = df['country'].str.extract(r'Town: (.*?)<br/>', expand=False)
# Country comes with two types of breaks. We need to extract the country from both types of breaks.
df['country'] = df['country'].str.extract(r'Country: (.*?)<br/>', expand=False)
df["country"] = df["country"].str.split("</p>").str[0]

# getting tender value in EUR.

df["value_eur"] = df.apply(convert_to_euro, axis=1)


# keep only relevant columns: id, year, title, cpv, contract_type, description, country, 
# buyer_name, buyer_address, buyer_town, contracting_authority_type, contracting_authority_activity, 
# is_framework_agreement, is_dynamic, is_open, value_eur

df = df[["id", "year", "title", "cpv", "contract_type", "description", "country", "buyer_name", "buyer_address", "buyer_town",
            "contracting_authority_type", "contracting_authority_activity", "government_procurement_agreement", "is_framework_agreement", "is_dynamic", "is_open", "value_eur"]]

In [4]:
df.head()

,id,year,title,cpv,contract_type,description,country,buyer_name,buyer_address,buyer_town,contracting_authority_type,contracting_authority_activity,government_procurement_agreement,is_framework_agreement,is_dynamic,is_open,value_eur
0,1-2021,2020,Cyber Threat Intelligence Infrastructure – Des...,[72000000],Services,ENISA seeks to contract service providers for ...,Greece,European Union Agency for Cybersecurity,1 Vasilissis Sofias Street,Maroussi,European institution/agency or international o...,Other,No,Yes,No,Yes,3150000
1,1000-2015,2014,Ds-rempart.,[72910000],Services,Maîtrise d'oeuvre du secours informatique de l...,France,Ministère de l'économie,Secrétariat général – SEP 1a – 18 avenue Léon ...,Paris Cedex 20,Ministry or any other national or federal auth...,General public services,Yes,No,No,Yes,2205140
2,100000-2015,2015,"Service de téléphonie filaire, de télécommunic...","[64210000, 64212000, 64221000, 72400000]",Services,"Services de téléphonie filaire, de télécommuni...",France,Communauté d'agglomération du Bassin d'Arcacho...,"2 allée d'Espagne, BP 147",Arcachon Cedex,Regional or local authority,Environment,Yes,Yes,No,Yes,NaN
3,100001-2014,2014,NaN,NaN,Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,NaN
4,100004-2015,2015,NaN,"[48810000, 48000000]",Services,Les prestations objet de la consultation conce...,France,Cerema,Bât. 4 — 25 avenue François Mitterrand — CS 92...,Bron Cedex,Body governed by public law,Environment,Yes,No,No,Yes,450000


# Duplicate check

Note: there will be no duplicates based on the id - these were removed already in the stage of html download.
Based on a manual check, current duplicates in the dataset are based on either a) rows that have all or many missing data b) tenders that were published multiple times as there were no suppliers.

In [4]:
# number of rows with missing values in all of: 
# title, description, contracting_authority_type, contracting_authority_activity, government_procurement_agreement, is_framework_agreement, is_dynamic, is_open, value_eur

df_missing = df[(df["title"].isna()) & (df["description"].isna())]

In [5]:
# show na in columns of df2

df_missing.isna().sum()

id                                      0
year                                    0
title                               21349
cpv                                 21347
contract_type                           2
description                         21349
country                             21347
buyer_name                          21347
buyer_address                       21347
buyer_town                          21347
contracting_authority_type          21347
contracting_authority_activity      21348
government_procurement_agreement    21348
is_framework_agreement              21347
is_dynamic                          21347
is_open                             21348
value_eur                           21343
dtype: int64

In [36]:
print("Number of rows with missing 80% of data: ", len(df[(df["title"].isna()) & (df["description"].isna())]))

Number of rows with missing 80% of data:  21349


In [39]:
# Drop rows with missing data in both title and description.

df = df.dropna(subset=["title", "description"], how="all")

In [40]:
duplicates = df[df.duplicated(subset=["title", "description", "country", "buyer_name", "buyer_address", "buyer_town", "value_eur", "year"])]

print("Number of rows that are duplicates: ", len(duplicates))

duplicates_sum = duplicates.groupby(["title", "description", "country", "buyer_name", "buyer_address", "buyer_town", "value_eur", "year"]).size().sort_values(ascending=False)
duplicates_sum

Number of rows that are duplicates:  26337


title                                                                                                                                                                                                                                                                                                                         description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [41]:
# droping duplicates

df = df.drop_duplicates(subset=["title", "description", "country", "buyer_name", "buyer_address", "buyer_town", "value_eur", "year"])

# Saving deduplicated data

In [42]:
# save cleaned df

df.to_csv('../data/cleaned_tender_data.csv', index=False)

# Simple value check

In [43]:
df = pd.read_csv('../data/cleaned_tender_data.csv')

print("Total number of records: ", len(df))

Total number of records:  301904


In [44]:
# for values in type of procedure, print unique values with their counts, including nan

df["is_open"].value_counts(dropna=False)

is_open
Yes    182955
No      71178
NaN     47771
Name: count, dtype: int64

In [33]:
df["contract_type"].value_counts(dropna=False)

Services    203879
Supplies     94670
Works         3250
NaN            105
Name: contract_type, dtype: int64

In [34]:
print(f"There are only {len(df[df['year'] == 2013])} records from 2013.")

df["year"].value_counts(dropna=False)

There are only 153 records from 2013.


2022    49023
2021    48680
2020    45370
2019    41535
2015    27461
2014    27286
2016    26717
2018    20591
2017    15088
2013      153
Name: year, dtype: int64

In [35]:
df["government_procurement_agreement"].value_counts(dropna=False)

Yes    162782
No      93280
NaN     45842
Name: government_procurement_agreement, dtype: int64

In [36]:
df["contracting_authority_type"].value_counts(dropna=False)

Body governed by public law                                  66593
Regional or local authority                                  61168
NaN                                                          52971
Other                                                        51717
Ministry or any other national or federal authority          48325
National or federal agency/office                            11810
Regional or local agency/office                               6023
European institution/agency or international organisation     3297
Name: contracting_authority_type, dtype: int64

In [37]:
# save value counts as csv

df["contracting_authority_activity"].value_counts(dropna=False)

General public services                                     90533
Other                                                       59855
NaN                                                         32689
Health                                                      29514
Education                                                   25041
Economic and financial affairs                              12206
Public order and safety                                      8498
Electricity                                                  6244
Environment                                                  5242
Defence                                                      4945
Social protection                                            4882
Housing and community amenities                              4145
Recreation culture and religion                              3676
Railway services                                             3660
Urban railway tramway trolleybus or bus services             3269
Water     

In [38]:
df["is_framework_agreement"].value_counts(dropna=False)

No     206701
Yes     54628
NaN     40575
Name: is_framework_agreement, dtype: int64

In [39]:
df["is_dynamic"].value_counts(dropna=False)

No     258345
NaN     40575
Yes      2984
Name: is_dynamic, dtype: int64

In [42]:
print(f"There are {len(df['country'].value_counts()[df['country'].value_counts() < 500])} countries with less than 500 records.")

print(df["country"].value_counts(dropna=False).to_string())



There are 56 countries with less than 500 records.
France                              40726
Germany                             36734
Poland                              34028
Spain                               28271
Czechia                             15317
United Kingdom                      14984
Netherlands                         13239
Sweden                              11744
Norway                              10970
Italy                                9247
Finland                              8497
Belgium                              7981
Romania                              6594
Lithuania                            6234
Hungary                              5331
Ireland                              5291
Denmark                              5157
Slovakia                             5150
Bulgaria                             4658
Switzerland                          4238
Portugal                             3709
Slovenia                             3638
Croatia                  